<a href="https://colab.research.google.com/github/guru3/soduku_solver/blob/master/3.%20One%20last%20time%3A%20Can%20a%20neural%20network%20solve%20the%20soduku%20game%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Let's give it another shot!

In [0]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"guru333","key":"695a60acdd9541c267fbd36712d5ffcd"}'}

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download bryanpark/sudoku
!unzip sudoku.zip

 95% 65.0M/68.1M [00:02<00:00, 20.6MB/s]
100% 68.1M/68.1M [00:02<00:00, 27.3MB/s]
Archive:  sudoku.zip
  inflating: sudoku.csv              


In [1]:
import csv
import random
import pickle
import numpy as np
from keras.callbacks import Callback
from keras.models import Sequential,Model
from keras.layers import Conv2D,Dense,Flatten,Input,Dropout
import matplotlib.pyplot as plt
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
def read_raw_data(total_to_read):
    FILE_PATH = './sudoku.csv';
    soduku_games = csv.reader(open(FILE_PATH,'r'))
    next(soduku_games);

    quizzes = [];
    solutions = [];
    index = 0;
    for game in soduku_games:
        index = index+ 1;
        if( index == total_to_read ):
            break;
        
        quizzes.append( np.reshape([int(d) for d in game[0]], (9, 9)) )
        solutions.append( np.reshape([int(d) for d in game[1]], (9, 9)) )
        
    permutation = np.random.permutation(len(quizzes));
    quizzes = np.array(quizzes)[permutation];
    solutions = np.array(solutions)[permutation];
    return quizzes, solutions;

In [0]:
def change_output(Y):
    return [ Y[:,i,j,:] for i in range(9) for j in range(9) ]

def remove_clues(inputArray, remove_clue=0):
    #very inefficient method - please perform walk of shame
    for inputs in inputArray:
      can_remove = [];
      for row in range(9):
        for col in range(9):
          if( np.sum(inputs[row,col,:]) != 0 ):
            can_remove.append( [row,col] )
      
      can_remove = np.array(can_remove)[ np.random.permutation(len(can_remove)) ];
      for i in range(remove_clue):
        [row,col] = can_remove[i];
        inputs[row,col,:] = 0;
    return inputArray;

def data_to_categorical(): # we will use one hot encoding of input and output  - this breakdown is more intuitive in the given case of discrete numbers
    X, Y = read_raw_data(20000);
    
    X = to_categorical(X, num_classes=10)
    X_also = to_categorical(Y, num_classes=10);
    Y = to_categorical(Y-1, num_classes=9)
    total_ex = X.shape[0];
    training_ex = (int)(total_ex*0.8);
    validation_ex= (int)(total_ex*0.1);
    train_X, train_Y = X_also[:training_ex], change_output(Y[:training_ex]);
    valid_X, valid_Y = X[training_ex:training_ex+validation_ex], change_output(Y[training_ex:training_ex+validation_ex]);
    test_X, test_Y = X[training_ex+validation_ex:], change_output(Y[training_ex+validation_ex:]);
    
    return X_also[:training_ex], train_Y, valid_X, valid_Y, test_X, test_Y;

In [0]:
train_X,train_Y,valid_X,valid_Y,test_X,test_Y = data_to_categorical();

In [5]:
print(train_X.shape)
print(len(train_Y),train_Y[0].shape)

(15999, 9, 9, 10)
81 (15999, 9)


In [0]:
def get_model():
    model = Sequential();
    model.add( Dense(128, input_shape=(9,9,10)) );
    model.add( Dropout(0.5) );
    model.add( Dense(64) );
    model.add( Dense(32) );
    model.add( Flatten() )
    
    input_ = Input(shape=(9,9,10))
    intermediate_output = model(input_)
    
    outputs = [ Dense(9, activation='sigmoid')(intermediate_output) for i in range(81) ] #for each cell in sudoku table
    model = Model(input_, outputs);
    model.compile( optimizer='RMSProp', loss='categorical_crossentropy', metrics= ['accuracy'] )
    return model

In [0]:
model = get_model();

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 9, 9, 10)     0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 2592)         11744       input_1[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 9)            23337       sequential_1[1][0]               
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 9)            23337       sequential_1[1][0]               
____________________________________________________________________________________________

In [0]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='loss', value=0.01, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

callbacks = [
    EarlyStoppingByLossVal(monitor='loss', value=1.0, verbose=1),
]

In [10]:
history = model.fit(train_X, train_Y, epochs=5000, batch_size=256, validation_data=(valid_X, valid_Y), callbacks=callbacks);1,

Train on 15999 samples, validate on 1999 samples
Epoch 1/5000
15999/15999 [==============================] - 20s 1ms/step - loss: 77.3737 - dense_4_loss: 0.9545 - dense_5_loss: 0.9608 - dense_6_loss: 0.9580 - dense_7_loss: 0.9255 - dense_8_loss: 0.9408 - dense_9_loss: 0.9319 - dense_10_loss: 0.9400 - dense_11_loss: 0.9505 - dense_12_loss: 0.9440 - dense_13_loss: 0.9425 - dense_14_loss: 0.9297 - dense_15_loss: 0.9419 - dense_16_loss: 0.9414 - dense_17_loss: 0.9546 - dense_18_loss: 0.9432 - dense_19_loss: 0.9414 - dense_20_loss: 0.9710 - dense_21_loss: 0.9216 - dense_22_loss: 0.9519 - dense_23_loss: 0.9726 - dense_24_loss: 0.9321 - dense_25_loss: 0.9429 - dense_26_loss: 0.9367 - dense_27_loss: 0.9472 - dense_28_loss: 0.9541 - dense_29_loss: 0.9378 - dense_30_loss: 0.9498 - dense_31_loss: 0.9385 - dense_32_loss: 0.9446 - dense_33_loss: 0.9265 - dense_34_loss: 0.9499 - dense_35_loss: 0.9541 - dense_36_loss: 0.9635 - dense_37_loss: 0.9408 - dense_38_loss: 0.9429 - dense_39_loss: 0.9591 - de

(1,)

In [12]:
for i in range(1,5):
  print('Training with missing clues ', i*3);
  train_X_round = remove_clues(train_X, remove_clue=3);
  history = model.fit(train_X_round, train_Y, epochs=5000, batch_size=128, validation_data=(valid_X, valid_Y), callbacks=callbacks);

Training with missing clues  3
Train on 15999 samples, validate on 1999 samples
Epoch 1/5000
15999/15999 [==============================] - 18s 1ms/step - loss: 2.5945 - dense_4_loss: 0.0370 - dense_5_loss: 0.0307 - dense_6_loss: 0.0354 - dense_7_loss: 0.0322 - dense_8_loss: 0.0321 - dense_9_loss: 0.0352 - dense_10_loss: 0.0341 - dense_11_loss: 0.0303 - dense_12_loss: 0.0322 - dense_13_loss: 0.0336 - dense_14_loss: 0.0293 - dense_15_loss: 0.0353 - dense_16_loss: 0.0296 - dense_17_loss: 0.0316 - dense_18_loss: 0.0319 - dense_19_loss: 0.0296 - dense_20_loss: 0.0308 - dense_21_loss: 0.0327 - dense_22_loss: 0.0299 - dense_23_loss: 0.0310 - dense_24_loss: 0.0328 - dense_25_loss: 0.0306 - dense_26_loss: 0.0304 - dense_27_loss: 0.0317 - dense_28_loss: 0.0347 - dense_29_loss: 0.0315 - dense_30_loss: 0.0364 - dense_31_loss: 0.0331 - dense_32_loss: 0.0302 - dense_33_loss: 0.0311 - dense_34_loss: 0.0328 - dense_35_loss: 0.0332 - dense_36_loss: 0.0371 - dense_37_loss: 0.0370 - dense_38_loss: 0.034

In [13]:
for i in range(1,5):
  print('Training with missing clues ', 12 + i*3);
  train_X_round = remove_clues(train_X, remove_clue=3);
  history = model.fit(train_X_round, train_Y, epochs=5000, batch_size=128, validation_data=(valid_X, valid_Y), callbacks=callbacks);

Training with missing clues  15
Train on 15999 samples, validate on 1999 samples
Epoch 1/5000
15999/15999 [==============================] - 18s 1ms/step - loss: 4.3927 - dense_4_loss: 0.0525 - dense_5_loss: 0.0518 - dense_6_loss: 0.0527 - dense_7_loss: 0.0544 - dense_8_loss: 0.0542 - dense_9_loss: 0.0476 - dense_10_loss: 0.0491 - dense_11_loss: 0.0522 - dense_12_loss: 0.0546 - dense_13_loss: 0.0540 - dense_14_loss: 0.0546 - dense_15_loss: 0.0547 - dense_16_loss: 0.0528 - dense_17_loss: 0.0578 - dense_18_loss: 0.0605 - dense_19_loss: 0.0552 - dense_20_loss: 0.0541 - dense_21_loss: 0.0543 - dense_22_loss: 0.0541 - dense_23_loss: 0.0563 - dense_24_loss: 0.0549 - dense_25_loss: 0.0594 - dense_26_loss: 0.0567 - dense_27_loss: 0.0550 - dense_28_loss: 0.0549 - dense_29_loss: 0.0465 - dense_30_loss: 0.0571 - dense_31_loss: 0.0521 - dense_32_loss: 0.0529 - dense_33_loss: 0.0519 - dense_34_loss: 0.0522 - dense_35_loss: 0.0524 - dense_36_loss: 0.0512 - dense_37_loss: 0.0492 - dense_38_loss: 0.04

KeyboardInterrupt: ignored

In [16]:
for test_case in range(5):
  output = model.predict(test_X[i:i+1])
  unsolved = np.array([ np.argmax(test_X[test_case,i,j,:]) for i in range(9) for j in range(9) ] ).reshape(9,9);
  print('Unsolved problem : \n', unsolved);
  solved = np.array([ (np.argmax(output[i]) + 1) for i in range(81) ]).reshape(9,9);
  print('Solution by model : \n', solved)
  actual_solution = np.array([ (np.argmax(np.array(test_Y)[i,test_case,:]) + 1 ) for i in range(81) ] ).reshape(9,9);
  print('Actual solution : \n',actual_solution)

Unsolved problem : 
 [[0 2 0 8 0 0 1 9 0]
 [0 7 1 0 0 5 6 3 4]
 [6 0 4 0 0 3 0 0 0]
 [0 0 0 0 3 6 0 5 0]
 [7 4 0 0 2 0 0 0 6]
 [5 0 0 9 0 8 2 0 1]
 [9 1 8 0 0 0 0 7 0]
 [0 0 3 0 0 0 9 0 2]
 [0 6 0 0 4 0 0 0 5]]
Solution by model : 
 [[5 3 8 5 3 1 9 1 6]
 [6 3 9 8 7 2 1 1 4]
 [8 1 9 9 6 2 3 5 2]
 [3 8 2 1 3 9 4 6 7]
 [1 9 6 4 5 5 1 2 5]
 [7 6 5 2 8 6 1 3 9]
 [2 2 4 6 1 3 5 9 9]
 [9 5 1 7 2 4 6 8 3]
 [2 6 3 5 9 5 2 6 1]]
Actual solution : 
 [[3 2 5 8 6 4 1 9 7]
 [8 7 1 2 9 5 6 3 4]
 [6 9 4 7 1 3 5 2 8]
 [1 8 2 4 3 6 7 5 9]
 [7 4 9 5 2 1 3 8 6]
 [5 3 6 9 7 8 2 4 1]
 [9 1 8 6 5 2 4 7 3]
 [4 5 3 1 8 7 9 6 2]
 [2 6 7 3 4 9 8 1 5]]
Unsolved problem : 
 [[0 0 0 6 0 1 0 0 0]
 [0 1 0 0 8 0 0 5 6]
 [0 9 0 3 0 2 0 0 0]
 [0 0 1 4 3 0 0 6 9]
 [0 0 0 2 0 5 0 3 8]
 [0 7 3 1 0 0 4 2 0]
 [0 2 9 0 0 7 0 0 0]
 [0 0 5 0 4 0 0 0 7]
 [0 0 6 9 0 0 0 4 1]]
Solution by model : 
 [[5 3 8 5 3 1 9 1 6]
 [6 3 9 8 7 2 1 1 4]
 [8 1 9 9 6 2 3 5 2]
 [3 8 2 1 3 9 4 6 7]
 [1 9 6 4 5 5 1 2 5]
 [7 6 5 2 8 6 1 3 9]
 [2 2 4 

### We do not see correct solutions, however we see that the numbers that have pre-set values do get properly mapped to the output. The issue is the values which are set to 0.

In [17]:
total_clues_removed = 24;
total_clue_to_remove = 61;
left_clues = 61 - 24;
train_X_round = remove_clues(train_X, remove_clue=37);
print('Total clues removed ', total_clue_to_remove);

Total clues removed  61


In [18]:
history = model.fit(train_X_round, train_Y, epochs=100, batch_size=128, validation_data=(valid_X, valid_Y), callbacks=callbacks);

Train on 15999 samples, validate on 1999 samples
Epoch 1/100
15999/15999 [==============================] - 19s 1ms/step - loss: 138.5553 - dense_4_loss: 1.7121 - dense_5_loss: 1.7087 - dense_6_loss: 1.7129 - dense_7_loss: 1.7181 - dense_8_loss: 1.7194 - dense_9_loss: 1.6773 - dense_10_loss: 1.6928 - dense_11_loss: 1.7007 - dense_12_loss: 1.7077 - dense_13_loss: 1.7299 - dense_14_loss: 1.7264 - dense_15_loss: 1.6892 - dense_16_loss: 1.7004 - dense_17_loss: 1.7133 - dense_18_loss: 1.7156 - dense_19_loss: 1.7078 - dense_20_loss: 1.6884 - dense_21_loss: 1.7319 - dense_22_loss: 1.6845 - dense_23_loss: 1.7079 - dense_24_loss: 1.7152 - dense_25_loss: 1.6938 - dense_26_loss: 1.6870 - dense_27_loss: 1.6993 - dense_28_loss: 1.7011 - dense_29_loss: 1.6911 - dense_30_loss: 1.7305 - dense_31_loss: 1.7145 - dense_32_loss: 1.7085 - dense_33_loss: 1.7046 - dense_34_loss: 1.6966 - dense_35_loss: 1.7025 - dense_36_loss: 1.7133 - dense_37_loss: 1.7006 - dense_38_loss: 1.7133 - dense_39_loss: 1.7133 - de

In [19]:
for test_case in range(5):
  output = model.predict(test_X[i:i+1])
  unsolved = np.array([ np.argmax(test_X[test_case,i,j,:]) for i in range(9) for j in range(9) ] ).reshape(9,9);
  print('Unsolved problem : \n', unsolved);
  solved = np.array([ (np.argmax(output[i]) + 1) for i in range(81) ]).reshape(9,9);
  print('Solution by model : \n', solved)
  actual_solution = np.array([ (np.argmax(np.array(test_Y)[i,test_case,:]) + 1 ) for i in range(81) ] ).reshape(9,9);
  print('Actual solution : \n',actual_solution)

Unsolved problem : 
 [[0 2 0 8 0 0 1 9 0]
 [0 7 1 0 0 5 6 3 4]
 [6 0 4 0 0 3 0 0 0]
 [0 0 0 0 3 6 0 5 0]
 [7 4 0 0 2 0 0 0 6]
 [5 0 0 9 0 8 2 0 1]
 [9 1 8 0 0 0 0 7 0]
 [0 0 3 0 0 0 9 0 2]
 [0 6 0 0 4 0 0 0 5]]
Solution by model : 
 [[5 2 8 3 5 2 9 1 6]
 [6 3 9 8 7 1 2 1 4]
 [9 1 9 9 6 2 3 5 2]
 [3 8 2 1 5 9 4 6 7]
 [1 9 6 4 5 6 1 2 5]
 [7 6 5 2 8 2 1 3 9]
 [8 2 4 6 1 3 5 9 9]
 [9 5 1 7 2 7 6 8 3]
 [2 2 3 5 9 5 2 6 1]]
Actual solution : 
 [[3 2 5 8 6 4 1 9 7]
 [8 7 1 2 9 5 6 3 4]
 [6 9 4 7 1 3 5 2 8]
 [1 8 2 4 3 6 7 5 9]
 [7 4 9 5 2 1 3 8 6]
 [5 3 6 9 7 8 2 4 1]
 [9 1 8 6 5 2 4 7 3]
 [4 5 3 1 8 7 9 6 2]
 [2 6 7 3 4 9 8 1 5]]
Unsolved problem : 
 [[0 0 0 6 0 1 0 0 0]
 [0 1 0 0 8 0 0 5 6]
 [0 9 0 3 0 2 0 0 0]
 [0 0 1 4 3 0 0 6 9]
 [0 0 0 2 0 5 0 3 8]
 [0 7 3 1 0 0 4 2 0]
 [0 2 9 0 0 7 0 0 0]
 [0 0 5 0 4 0 0 0 7]
 [0 0 6 9 0 0 0 4 1]]
Solution by model : 
 [[5 2 8 3 5 2 9 1 6]
 [6 3 9 8 7 1 2 1 4]
 [9 1 9 9 6 2 3 5 2]
 [3 8 2 1 5 9 4 6 7]
 [1 9 6 4 5 6 1 2 5]
 [7 6 5 2 8 2 1 3 9]
 [8 2 4 

#### We see better solutions now! How can we improve it going forward?
#### We should add constraint in the model optimization itself, to have different numbers in each row, column and grid! That should direct the network into more concrete path towards optimal solution.